In [2]:
import numpy as np
import cv2
from skimage import exposure

In [3]:
def load_image(filename):
    image = cv2.imread(filename, cv2.IMREAD_ANYDEPTH)
    return image
 
def is_inverted(image, corner_ratio=0.2, center_ratio=0.01, bottom=True):
    height, width = image.shape
    corner_size = (int(height*corner_ratio), int(width*corner_ratio))
    tl = image[:corner_size[0], :corner_size[1]].flatten()
    bl = image[-corner_size[0]:, :corner_size[1]].flatten()
    tr = image[:corner_size[0], -corner_size[1]:].flatten()
    br = image[-corner_size[0]:, -corner_size[1]:].flatten()
    if bottom:
        corners = np.hstack([tl, bl, tr, br])
    else:
        corners = np.hstack([tl, tr])
            
    center = image[
        int(height*(0.5-center_ratio)):int(height*(0.5+center_ratio)),
        int(width*(0.5-center_ratio)):int(width*(0.5+center_ratio))
    ]
    return center.mean() < corners.mean()

### ideas for cutting the frame

In [ ]:
def cut_frames(image, max_frame_ratio=0.2):
    height, width = image.shape
    img = image[:, int(width*max_frame_ratio):int(width*(1-max_frame_ratio))]
    vert_mask = img.mean(1)*0.01 > np.abs(np.mean(img - img.mean(1)[:, None],1))
    img = image[int(height*max_frame_ratio):int(height*(1-max_frame_ratio))]
    horiz_mask = img.mean(0)*0.01 > np.abs(np.mean(img - img.mean(0),0))
    return image[vert_mask][:, horiz_mask]

def cut_frames2(image, max_frame_ratio=0.2, mode_threshold = 0.2):
    height, width = image.shape
    vert_mask = stats.mode(image, axis=1)[1].flatten() < width * mode_threshold
    horiz_mask = stats.mode(image, axis=0)[1].flatten() < height * mode_threshold
    print(horiz_mask)
    return image[vert_mask][:, horiz_mask]

def normalize(image):
    return 